# Veri Önişleme

In [4]:
import os

In [5]:
import json

path = '..\\veri'

def yukle():
    liste = os.listdir(path)
    print(liste)
    final_path = path + "\\" + liste[-1]
    print(final_path)
    with open(final_path, encoding='utf-8') as f:
        veri = json.load(f)
    return veri

In [6]:
df_ = yukle()

['.ipynb_checkpoints', 'ankarabbSoruCevapVeriSeti-ham.xlsx', 'ankarabbSoruCevapVeriSeti-v01.xlsx', 'ankarabbSoruCevapVeriSeti-v02.xlsx', 'ankarabbSoruCevapVeriSetiJson-v02.json', 'ankarabbSoruCevapVeriSetiJson-v10.json', 'ankarabbSoruCevapVeriSetiJson-v11.json']
..\veri\ankarabbSoruCevapVeriSetiJson-v11.json


In [7]:
df_

{'Tüm Sorular': [{'tag': 'ankara_01',
   'patterns': ['ÖĞRENCİ ABONMAN KARTINDAN YARARLANMA ŞARTLARI NELERDİR?',
    'Öğrenciler için abonman kart şartları nedir',
    'Abonman kart öğrenci şartları',
    'Abonman kart için öğrenciler ne yapmalıdır',
    'İndirimli Abonman Kart için öğrencilerin yapması gerekenler'],
   'responses': ['1. Ankara Büyükşehir Belediye Başkanımız Sayın Mansur YAVAŞ’ın öğrencilerimize destek sağlamak amacıyla başlattığı, Aktif Öğrenci (İndirimli) Ankarakart’lara uygulanan abonman uygulaması, 18.10.2019 tarih ve 2019/87 E. sayılı UKOME Kararı ile başlatılmıştır.\n\n2. Öğrenci Abonman uygulaması, 60 TL karşılığında 200 biniş hakkı verir. Yapıldığı tarihten itibaren 30 gün geçerlidir ve bir sonraki ayın aynı günü, aynı saatte sona erer.\n\n3. Abonman uygulaması şehir içi hizmet veren EGO, Özel Halk Otobüsleri (ÖHO), Özel Toplu Taşıma Araçları (ÖTA), Metro, Ankaray ve Teleferik’te geçerlidir (Tam Ankarakart 3,25 TL, Öğrenci Ankarakart 1,75 TL olarak uygulanan). 

In [8]:
from unicode_tr import unicode_tr
from snowballstemmer import TurkishStemmer
from nltk import ngrams
from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")


def metin_onisle(metin):
    kucuk_harfli_metin = unicode_tr(metin).lower()
    istenen_karakterler = set(list(' abcdefghijklmnopqrstuvwxyzâçîöüğış0123456789'))
    harfler = list(kucuk_harfli_metin)
    harfler = [k if k in istenen_karakterler else ' ' for k in harfler]
    temiz_dokuman = "".join(kucuk_harfli_metin)
    temiz_dokuman_kelimeleri = temiz_dokuman.split(' ')
    temiz_dokuman_kelimeleri = [kelime for kelime in temiz_dokuman_kelimeleri if len(kelime) > 0]
    stopWords = set(stopwords.words('turkish'))
    temiz_dokuman_kelimeleri = ["" if kelime in stopWords else kelime for kelime in temiz_dokuman_kelimeleri]
    temiz_dokuman_kelimeleri = " ".join(temiz_dokuman_kelimeleri).replace("  ", " ").split()
    turkStem=TurkishStemmer()
    temiz_dokuman_kelimeleri = [turkStem.stemWord(kelime) for kelime in temiz_dokuman_kelimeleri]
    n = 2
    bigrams = ngrams(temiz_dokuman_kelimeleri, n)
    bigramstr = map(''.join, bigrams)
    ngram = " ".join(list(bigramstr))
    temiz_dokuman = " ".join(temiz_dokuman_kelimeleri) + " " +  "".join(ngram)
    return temiz_dokuman

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alierenekinci\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
for veri_ in df_:
    for i, j in enumerate(df_[veri_]):
        for k,l in enumerate(j["patterns"]):
            df_[veri_][i]["patterns"][k] = metin_onisle(l)
        # for k,l in enumerate(j["responses"]):
        #    df_[veri_][i]["responses"][k] = metin_onisle(l)

In [10]:
df_

{'Tüm Sorular': [{'tag': 'ankara_01',
   'patterns': ['öğrenci abonma kart yararlanma şart nelerdir? öğrenciabonma abonmakart kartyararlanma yararlanmaşart şartnelerdir?',
    'öğrenci abonma kart şart ne öğrenciabonma abonmakart kartşart şartne',
    'abonma kart öğrenci şart abonmakart kartöğrenci öğrencişart',
    'abonma kart öğrenci yapmal abonmakart kartöğrenci öğrenciyapmal',
    'indirimli abonma kart öğrenci yapmas gereken indirimliabonma abonmakart kartöğrenci öğrenciyapmas yapmasgereken'],
   'responses': ['1. Ankara Büyükşehir Belediye Başkanımız Sayın Mansur YAVAŞ’ın öğrencilerimize destek sağlamak amacıyla başlattığı, Aktif Öğrenci (İndirimli) Ankarakart’lara uygulanan abonman uygulaması, 18.10.2019 tarih ve 2019/87 E. sayılı UKOME Kararı ile başlatılmıştır.\n\n2. Öğrenci Abonman uygulaması, 60 TL karşılığında 200 biniş hakkı verir. Yapıldığı tarihten itibaren 30 gün geçerlidir ve bir sonraki ayın aynı günü, aynı saatte sona erer.\n\n3. Abonman uygulaması şehir içi hizmet

In [11]:
def kaydet(veri):
    liste = os.listdir(path)
    final_path = path + "\\" + liste[-1][:-7] + str(int(liste[-1][31:-5])+1) + "-onislenmis" + ".json"
    with open(final_path, 'w', encoding='utf-8') as json_dosya:
        json.dump(veri, json_dosya, ensure_ascii=False)

    print("Dosya adı:", final_path)

In [12]:
kaydet(df_)

Dosya adı: ..\veri\ankarabbSoruCevapVeriSetiJson-v12-onislenmis.json
